In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv(r"unique_movie_dataset(2).csv")
movies = movies.drop(columns=['Unnamed: 0','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'])
movies.head()

,unique_original_title,unique_id
0,Trois couleurs : Rouge,tt0111495
1,Les Quatre Cents Coups,tt0053198
2,Sleepless in Seattle,tt0108160
3,Rocky Balboa,tt0479143
4,Fools Rush In,tt0119141


# In the unique_movie_dataset, we have the 'unique_id' column, which is actually the id of the movie assigned to it by imdb and which is present in the url as soon as we search for the movie on imdb.This id will helps us to scrape the reviews of the movie easily from imdb by understanding the structure of url and performing scrapping accordingly.

In [3]:
movies.head()

,unique_original_title,unique_id
0,Trois couleurs : Rouge,tt0111495
1,Les Quatre Cents Coups,tt0053198
2,Sleepless in Seattle,tt0108160
3,Rocky Balboa,tt0479143
4,Fools Rush In,tt0119141


## Reviews Scrapping Code:

In [4]:
# This code is a Python script that scrapes movie reviews from the IMDb website and saves them into
# an Excel file. Here's a breakdown of what each line does:



# Importing required python libraries,pandas: for data manipulation,requests: for making HTTP 
# requests to the IMDb website and BeautifulSoup: for parsing HTML content.

import pandas as pd
import requests
from bs4 import BeautifulSoup

# this is a fnc that takes a single argument 'movie_id' as function parameter which is actually
# the unique id of the movie provided in the IMDb url.

def scrape_reviews(movie_id):
    
    # initializing an empty list 'reviews' , which will later be used to store the scraped reviews. 
    reviews = []
    
    # If we observe the IMDb url structure,on searching for different movies the movie_id is the
    # part of the url that continuously changes and therefore we declare a variable 'url' that
    # will contain the IMDb url for the movie reviews page,with the {} placeholder replaced by 
    # the movie_id argument.
    url = 'https://www.imdb.com/title/{}/reviews/_ajax?ref_=undefined&paginationKey='.format(movie_id)
    
    # pagination_key is a variable that will be used to retrieve addtional pages of reviews because
    # by default only 25 reviews are displayed per page in the IMDb review section and we want
    # 100 reviews per movie for our dataset.    
    pagination_key = ''
    
    # So we start a while loop that will run until our review list length is less than 100.     
    while len(reviews) < 100:
        
        # Here, we make an HTTP get request to the url variable to get the HTML content of the url
        # and the 'pagination_key' variable is appended to the end of the url to retrieve the content
        # of the specific pagination key, which is further used to access additional pages of reviews
        # beyond the first page.     
        response = requests.get(url + pagination_key)
        
        # creating BeautifulSoup object from the HTML content retrieved from the IMDb website.         
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # This loop iterates through all the 'div' elements with the 'text' class,which contain 
        # movie reviews.Then we strip leading & trailing whitespaces from the review & append it
        # to the review list.As soon as the length of list==100 we break from the loop.         
        for review in soup.find_all('div', {'class': 'text'}):
            review_text = review.get_text().strip()
            reviews.append(review_text)
            if len(reviews) == 100:
                break
                
        # This finds the div element with the load-more-data class, which contains a data-key
        # attribute that can be used to retrieve additional pages of reviews. If this element 
        # exists, its data-key value is assigned to the pagination_key variable, which is then 
        # appended to the URL for the next page of reviews. If the element does not exist, the loop
        # is exited with a break statement.        
        pagination_key_element = soup.find('div', {'class': 'load-more-data'})
        if pagination_key_element:
            pagination_key = pagination_key_element.get('data-key')
        else:
            break
            
    return reviews

# Create a dictionary to store the reviews for each movie
reviews_dict = {}

# Loop through each movie in the DataFrame and call the scrape_reviews() function to get the review
# The iterrows() function allows us to iterate over the rows of a Pandas dataframe as (index, Series) pairs.
for index, row in movies.iterrows():
    movie_id = row['unique_id']
    reviews = scrape_reviews(movie_id)
    reviews_dict[movie_id] = reviews

# Convert the dictionary to a new DataFrame and export it to a new Excel file
reviews_df = pd.DataFrame.from_dict(reviews_dict, orient='index')
reviews_df.to_excel('reviews_list.xlsx')

In [5]:
# Reviews Dataframe: 

pd.set_option('display.max_columns',None)
reviews_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
tt0111495,It's almost impossible for me to sit down and ...,This is the last film of Krzysztof Kieslowski ...,"The last film in the Three Colors trilogy, RED...",The final and most haunting of Polish director...,"Valentine, a model in France is separated from...",Trilogies are very interesting. Some go out wi...,One of my favorite films of all time. With bea...,The final part of Kieslowski's trilogy based o...,"In my analysis of ""Trois couleurs: Blanc"" I wr...","I found ""Three Colors: Red"" to be a strangely ...","""Three Colours: Red"" is an out-of-body movie e...",See Three Colors: Blue and Three Colors: White...,"A young fashion model, Valentine, is in a long...","Reasonably Intriguing, but slow-moving with a ...",If you want to watch just one Krzysztof Kieslo...,I enjoyed all three of the movies in this seri...,Kieslowski gives us a tapestry of interwoven s...,"""Red"" (French, 1994): Exquisite, thoughtful dr...","All three films in the ""Three Colours Trilogy""...",Spoilers herein.In addition - or perhaps in sp...,Valentine Dussaut (Irène Jacob) is a young mod...,"I feel at a loss, so brilliant is this film. ...",British critics were always envious as to how ...,It is not only difficult to comment separately...,"""Rouge"" is part of a trilogy, but very much st...","This, the third of Krzysztof Kieslowski 'Three...","I find Blue, the first film in this loose, the...","In the ""goofs"" section for this film there's a...",Having seen White before I realized it was a t...,"I saw ""Red"" years ago and I still have not fou...","In its own quiet way, ""Red"" is one of the most...",The Swiss/Polish/French film Trois couleurs: R...,Loved the movie and the story. When a movie is...,'Rouge' is the last part from the 'Trois Coule...,Its my favourite film because there's so much ...,"More than a great film, last part of trilogy, ...",For years I've been running around in countles...,Third outing in the thought-provoking and broo...,Irene Jacob is mesmerizing in this final insta...,Red is the third and final film in the Three C...,"I had always heard the ""color"" movies of Polis...","and there are not many in cinema history. ""Rou...","Kieslowski, IMHO, is one of the 3 best directo...",What a film. It needs the talent of a maestro ...,"A beautiful passionate film, the most beautifu...","I heard about these films, and the result of t...",There's at least one certainty to gather from ...,What's the fuss about? I am going to be the on...,"Three Colours: Red stars ""The Double Life of V...",The last and most critically acclaimed film of...,"Master Kieslowsky came with an idea in 1993, t...",I'm surprised how this movie got amazing revie...,The third film in Krzysztof Kieslowski's Three...,I was told to watch this amazing film. I neede...,This is definitely my favorite within the Thre...,I've just been reading some of the comments po...,Are we allowed to interfere with our fellowmen...,"""Three Colors: Red"" follows a model named Vale...",My Rating : 9/10Three Colors: Red is a unique ...,"This is a classic continuation to Bleu, the li...","It's kind of likeable, the protagonist actress...",It's a personal drama about the intersection o...,"This is a sometimes clever, sometimes corny, b...",The third installment of Krzysztof Kieslowski'...,"When I saw Blue about 6 months ago, it went st...",Three Colors: Red takes its audience on an enc...,The hardest of the three to get into. It lacks...,Most of Kieslowski's films seem like puzzles t...,Their is something open and heartwarming about...,Three colours red is by far the best film of K...,How often do we see a film that resides within...,The film is about unlikely friendship. The che...,The finest trilogy

In [6]:
# Extracting reviews in a csv file: 

file_path = r'C:\Users\KIIT\Desktop\MINOR_PROJECT\Datasets\movie_id_reviews(2).csv'
reviews_df.to_csv(file_path,index=True)